In [12]:
from datetime import datetime,timedelta
import time
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
import MetaTrader5 as mt5
import yfinance as yf
import mplfinance as mpf
from plotly.subplots import make_subplots

In [13]:
class MT5:
    def get_data(symbol,n, timeframe=mt5.TIMEFRAME_D1):
        """Function to import the data of a chosen symbol"""
        
        # intitialize the connection if there is not
        if not mt5.initialize():
            print('initialize() failed, error code :',mt5.last_error())
            mt5.shutdown()
            
        # current date extract
        utc_from= datetime.now()
        
        # Import the data into a tuple
        rates= mt5.copy_rates_from(symbol, timeframe, utc_from,n)
        
        # Tuple to dataframe
        rates_frame= pd.DataFrame(rates)
        
        #convert time in seconds into the datetime format
        rates_frame['time']= pd.to_datetime(rates_frame['time'],unit='s')
        
        #convert the column 'time' in the right format
        rates_frame['time']=pd.to_datetime(rates_frame['time'], format='%Y-%m-%d')
        
        #set column time as the index of the dataframe
        rates_frame = rates_frame.set_index('time')
        
        return rates_frame
    
    
    def orders(symbol, lot, buy=True, id_position=None):
        """Send the orders"""
        
        # Initialize the connection if there is not
        if not mt5.initialize():
            print('initialize() failed, error code :',mt5.last_error())
            mt5.shutdown()
        # Get filling mode
        filling_mode= mt5.symbol_info(symbol).filling_mode-1
        
        # Take ask price
        ask_price= mt5.symbol_info_tick(symbol).ask
        
        # Take bid price
        bid_price=mt5.symbol_info_tick(symbol).bid
        
        #Take the point of the asset
        point = mt5.symbol_info(symbol).point
        
        deviation= 20
        
 # ****************open a trade ****************
        if id_position == None:
            
            # Buy order parameters
            if buy:
                type_trade= mt5.ORDER_TYPE_BUY
                sl= ask_price*(1-0.01)
                tp=ask_price*(1+0.01)
                price= ask_price
                
            # sell order parameters
            else:
                type_trade=mt5.ORDER_TYPE_SELL
                sl=bid_price*(1+0.01)
                tp=bid_price*(1-0.01)
                price=bid_price
                
            #open the trade
            
            request={
               "action":mt5.TRADE_ACTION_DEAL,
               "symbol": symbol,
               "volume": lot,
               "type": type_trade,
               "price": price,
               "deviation": deviation,
               "sl": sl,
               "tp": tp,
               "magic": 234000,
               "comment": "python script order",
               "type_time": mt5.ORDER_TIME_GTC,
               "type_filling": filling_mode
            
            
            }
        
            # send a trading request
            result=mt5.order_send(request)
            result_comment= result.comment
        
    #****************close a trade**************
        else:
            # Buy order Parameters
            if buy:
                type_trade= mt5.ORDER_TYPE_SELL
                price = bid_price
                
            # sell order parameters
            else:
                type_trade= mt5.ORDER_TYPE_BUY
                price = ask_price
                
            #close the trade
            request={
               "action":mt5.TRADE_ACTION_DEAL,
               "symbol": symbol,
               "volume": lot,
               "type": type_trade,
               "position": id_position,
               "price": price,
               "deviation": deviation,
               "magic": 234000,
               "comment": "python script order",
               "type_time": mt5.ORDER_TIME_GTC,
               "type_filling": filling_mode
            }
            
            # send a trading request
            result=mt5.order_send(request)
            result_comment= result.comment

        return result_comment
     
    def resume():
        """ Retuen the current positions. Position=0 --> Buy"""
        # Initialize the connection if there is not
        if not mt5.initialize():
            print('initialize() failed, error code :',mt5.last_error())
            mt5.shutdown()
            
        # Define the name of the columns that we will create
        columns=['ticket', 'position','symbol','volume']
        
        # Go take the current open trades
        current= mt5.positions_get()
        
        # Create a empty dataframe
        summary = pd.DataFrame()
        
        #Loop to add each row in dataframe
        for element in current:
            element_pandas= pd.DataFrame([element.ticket,
                                          element.type,
                                          element.symbol,
                                          element.volume],
                                         index=columns).transpose()
            summary= pd.concat((summary, element_pandas), axis=0)
            
        return summary
    
    
    def run(symbol, long, short, lot):
        # Initialize
        if mt5.initialize()== False:
            mt5.initialize()
            
        # Choose your symbol
        print("-----------------------------------")
        print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print("SYMBOL",symbol)
        
        # Initialize the device
        current_open_positions =MT5.resume()
        # Buy or sell
        print(f"BUY:{long}\t SHORT:{short}")
        
        """Close trade eventually"""
        
        # Extraction type trade
        
        try:
            position= current_open_positions.loc[current_open_positions['symbol']==symbol].values[0][1]
            
            identifier= current_open_positions.loc[current_open_positions['symbol']==symbol].values[0][0]
        except:
            position= None
            identifier= None
            
        # Close trades
        if long == True and position ==0:
            long= False
            
        elif long == False and position ==0:
            res=MT5.orders(symbol, lot, buy=True, id_position=identifier)
            print(f"CLOSE LONG TRADE: {res}")
            
        elif short == True and position==1:
            short=False
            
        elif short ==False and position ==1:
            res= MT5.orders(symbol, lot, buy=False, id_position=identifier)
            print(f"CLOSE SHORT TRADE:{res}")
            
        else:
            pass
        
        """Buy or short"""
        
        if long== True:
            res= MT5.orders(symbol,lot, buy=True, id_position=None)
            print(f"OPEN LONG TRADE:{res}")
            
        if short == True:
            res= MT5.orders(symbol,lot, buy=False, id_position=None)
            print(f"OPEN SHORT TRADE:{res}")
            
        
        
        
            


In [5]:
 def random():
    values=[True, False]
    buy=np.random.choice(values)
    sell= not buy
            
    return buy,sell

symbols_list={
    "Gold USD":["XAUUSD",0.01],
    "S&P 500":["US500",0.01],
    "Us dollar vs Canadian dollar":["USDCAD",0.01],
    "Euro vs Usdollar":["EURUSD",0.01]}

current_account_info= mt5.account_info()
print("----------------------------------------------------")
print(f"Date:{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Balance:{current_account_info.balance}USD,\t"
      f"Equity:{current_account_info.equity}USD,\t"
      f"Profit:{current_account_info.profit}USD,\t")
print("----------------------------------------------------")


start= datetime.now().strftime("%H:%M:%S")
      
while True:
    # Launch the algorithm
    # Verification for launch
    if datetime.now().weekday() not in (5,6):
        is_time=datetime.now().strftime("%H:%M:%S") == start
    else:
        is_time=False
        
    if is_time:
        #close all trades
        if mt5.positions_total()>0:
            MT5.close_all_night()
            
        for asset in symbols_list.keys():
            #Initialize the inputs
            symbol=symbols_list[asset][0]
            lot=symbols_list[asset][1]
            
            buy,sell=random()
            
            #Run the algorithm
            MT5.run(symbol,buy,sell,lot)
    

In [10]:
mt5.TIMEFRAME_M10